In [110]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer 
from transformers.generation import GenerationConfig
torch.manual_seed(1234)

from IPython.display import Latex, Audio

In [3]:
from tqdm.auto import tqdm

In [4]:
import pandas as pd
import os

In [5]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig
import random

In [6]:
DEVICE = torch.device('cuda:1')

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-Audio", trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map="cpu", trust_remote_code=True).eval()
# use cuda device
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio", device_map=DEVICE, trust_remote_code=True).eval()

eos_token_id = tokenizer('<|endoftext|>',return_tensors='pt').input_ids[0]

audio_start_id: 155163, audio_end_id: 155164, audio_pad_id: 151851.


In [111]:
path_to_ckpts = 'ckpts/asr-sentence-full-llm/version_6/'

In [112]:
peft_model_id = os.path.join(path_to_ckpts, "tuned-model/")
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, torch_dtype=torch.bfloat16, device_map=DEVICE, trust_remote_code=True)


model = PeftModel.from_pretrained(model, peft_model_id, device_map=DEVICE)
_ = model.eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [115]:
test =pd.read_csv('test_ENG.csv')
test.head()

,speaker_id,language,formula_id,is_tts,pronunciation,whisper_transcription,latex,audio_path
0,0,eng,4178,0,mu minus nu hat.,Mu hat minus Nu hat.,\hat \mu-\hat \nu,/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
1,0,eng,4231,0,mu tilde is defined as the set of mu one plus ...,MuTiled is defined as the set of Mu 1 plus Mu...,\tilde { \mu } : = \ { \mu_1 + \tilde { \mu } ...,/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
2,0,eng,4321,0,N sub G of s sub 1 union N sub G of s sub 2 se...,"NG union NG set- open brace C1, C2 close brace",N_G ( s_1 ) \cup N_G ( s_2 ) \setminus \ { s_1...,/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
3,0,eng,4374,0,"N, open parenthesis, 1 sub Q transpose, comma,...",n 1 sub q transpose comma 4 sub i sub n close...,"\mathcal { N } \left ( { 1_Q^ \top,4 { I_n } }...",/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...
4,0,eng,4507,0,Negative zeta to the power of negative one-hal...,negative zeta to the power of negative one ha...,"\mathfrak { C } = [ -\zeta^ { -1/2 } , \zeta^ ...",/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita...


In [116]:
gen_params = {
    "max_new_tokens": 200,
    # "repetition_penalty": 1.0,
    "remove_invalid_values": True,
    "eos_token_id": eos_token_id,
    "forced_eos_token_id": eos_token_id,
    "use_cache": True,
    # "no_repeat_ngram_size": 4,
    # "do_sample": False,
    "num_return_sequences": 1,
    }

In [135]:
# audio_url = "../2646.wav"
# audio_url = "../4763.wav"
N = random.randint(0, len(test))
audio_url = test.iloc[N]['audio_path']
true_latex = test.iloc[N]['latex']
sp_prompt = "<|startofanalysis|><|en|><|caption|><|en|><|notimestamps|><|wo_itn|>"
query = f"<audio>{audio_url}</audio>{sp_prompt}"
audio_info = tokenizer.process_audio(query)
inputs = tokenizer(query, return_tensors='pt', audio_info=audio_info)

In [136]:
inputs = inputs.to(model.device)
pred = model.generate(**inputs, **gen_params, audio_info=audio_info)
response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False, audio_info=audio_info)
print(response)
wo_ind = response.find('<|wo_itn|>')
response = response[wo_ind:].replace('<|endoftext|>', '').replace('<|wo_itn|>', '')
print(response)

<audio>/workspace-SR006.nfs2/shares/SR006.nfs2/Nikita/speech2latex/saluth__mathbridge_eng/audio_8316.wav</audio><|startofanalysis|><|en|><|caption|><|en|><|notimestamps|><|wo_itn|>2 \div 0.02 + 0.1 = 100<|endoftext|>
2 \div 0.02 + 0.1 = 100


In [137]:
Latex(f'${response}$')

<IPython.core.display.Latex object>

In [134]:
Latex(f'${true_latex}$')

<IPython.core.display.Latex object>

In [ ]:
outputs = []
for i in tqdm(range(len(test))):
    audio_url = test.iloc[i]['audio_path']
    true_latex = test.iloc[i]['latex']
    sp_prompt = "<|startofanalysis|><|en|><|caption|><|en|><|notimestamps|><|wo_itn|>"
    query = f"<audio>{audio_url}</audio>{sp_prompt}"
    audio_info = tokenizer.process_audio(query)
    inputs = tokenizer(query, return_tensors='pt', audio_info=audio_info)
    inputs = inputs.to(model.device)
    pred = model.generate(**inputs, **gen_params, audio_info=audio_info)
    response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=False, audio_info=audio_info)
    # print(response)
    wo_ind = response.find('<|wo_itn|>')
    response = response[wo_ind:].replace('<|endoftext|>', '').replace('<|wo_itn|>', '')
    # print(response)

    outputs.append({
        'pred': response,
        'true_latex': true_latex
    })

  0%|          | 0/2594 [00:00<?, ?it/s]

In [166]:
n = random.randint(0, len(outputs))
pred, true = outputs[n]['pred'], outputs[n]['true_latex']

In [167]:
Latex(f'${pred}$')

<IPython.core.display.Latex object>

In [168]:
Latex(f'${true}$')

<IPython.core.display.Latex object>

In [169]:
len(outputs)

1270

In [108]:
torch.save(outputs, 'outputs_lora_1_epoch_lora_audio.pth')

In [139]:
print()

In [ ]:
torch.load('outputs_lora_1_epoch_lora_audio.pth')